In [3]:
import pandas as pd

df = pd.read_csv('C:/Users/Admin/Desktop/olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


Question 0 (Example)

In [4]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

Question 1

In [5]:
#Which country has won the most gold medals in summer games?
#This function should return a single string value.

def answer_one():
    return df[df['Gold'] == max(df['Gold'])].index[0]
answer_one()

'United States'

Question 2

In [6]:
# Which country had the biggest difference between their summer and winter gold medal counts?
# This function should return a single string value.

def answer_two():
    df['difference'] = abs(df['Gold']-df['Gold.1'])
    return df[df['difference'] == max(df['difference'])].index[0]
answer_two()

'United States'

Question 3

In [14]:
# Which country has the biggest difference between their summer and winter gold medal counts relative to their total gold medal count? 
# Only include countries that have won at least 1 gold in both summer and winter.
# This function should return a single string value.

def answer_three():
    a = df.copy()
    a = a.drop(a[a['Gold'].isin([0])&a['Gold.1'].isin([0])].index)  
    a = a[(a['Gold.1'] != 0) & (a['Gold'] != 0)]
    a['relative'] = abs((a['Gold']-a['Gold.1'])/a['Gold.2'])
    return a[a['relative'] == max(a['relative'])].index[0]
answer_three()

'Bulgaria'

Question 4

In [9]:
# Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. 
# The function should return only the column (a Series object) which you created.
# This function should return a Series named Points of length 146

def answer_four():
    df['Points'] = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']*1
    return df['Points']
answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64

Question 5

In [10]:
# Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

census_df = pd.read_csv('C:/Users/Admin/Desktop/census.csv')
census_df.head()

def answer_five():
    df=census_df[census_df['SUMLEV'] == 50]
    df = df.groupby( [ "SUMLEV", "STNAME"] ).size().to_frame(name = 'count').reset_index()
    return df.loc[df['count'].idxmax()]['STNAME']
answer_five()  

'Texas'

Question 6

In [11]:
# Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

def answer_six():
    census_copy_df=census_df.drop(census_df[census_df['COUNTY']==0].index)
    state =list(census_copy_df['STNAME'].unique())
    counts = dict()
    for item in state:
        counts[item] = sum(census_copy_df.where(census_copy_df['STNAME'] == item).nlargest(3,'CENSUS2010POP')['CENSUS2010POP'])
    return list(pd.Series(counts).nlargest(3).index)
answer_six()

['California', 'Texas', 'Illinois']

Question 7

In [12]:
def answer_seven():
    census_copy=census_df.drop(census_df[census_df['COUNTY']==0].index)
    census_copy['max_dif']=census_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)-census_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    return list(census_copy[census_copy['max_dif'] == max(census_copy['max_dif'])]['CTYNAME'])[0]  #.astype(str)
answer_seven()

'Harris County'

Question 8

In [26]:
def answer_eight():
    df=census_df[census_df['SUMLEV'] == 50]
    region1 = df['REGION'] == 1
    region2 = df['REGION'] == 2
    county_name = df['CTYNAME'].str.startswith('Washington')
    pop_estim = df['POPESTIMATE2015'] > df['POPESTIMATE2014']
    df = df[(region2|region1)&county_name&pop_estim]
    return df[['STNAME', 'CTYNAME']]
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
